In [1]:
import pandas as pd
import tensorflow as tf
import os
import shutil
import numpy as np
from tqdm.auto import tqdm

2023-08-17 22:24:37.121372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 22:24:37.913218: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATASET_DIR = "../data/asl-fingerspelling"
TRAIN_LANDMARKS_PATH = DATASET_DIR+os.sep+"train_landmarks"
SUPPLEMENTAL_LANDMARKS_PATH = DATASET_DIR+os.sep+"supplemental_landmarks"
TRAIN_PATH = DATASET_DIR+os.sep+"train.csv"
TFRECORDS_PATH = DATASET_DIR+os.sep+"tfrecords"

In [3]:
#Dataset data frame

train_df = pd.read_csv(TRAIN_PATH)

In [4]:
train_df.head()

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


In [12]:
train_df.sort_values(by="sequence_id")

,path,file_id,sequence_id,participant_id,phrase
19273,train_landmarks/566963657.parquet,566963657,71095,56,star-shop/anabarreto.loja.9
19274,train_landmarks/566963657.parquet,566963657,71689,216,www.teteeaddict.com/ec-market
19275,train_landmarks/566963657.parquet,566963657,95670,36,9310 bertland road
19276,train_landmarks/566963657.parquet,566963657,158691,81,5790 saint dirt
19277,train_landmarks/566963657.parquet,566963657,189294,76,9703 rojean southwest
...,...,...,...,...,...
13271,train_landmarks/450474571.parquet,450474571,2147338020,20,slworld.net/composers
13272,train_landmarks/450474571.parquet,450474571,2147373562,219,+66-979-32-8896-827
13273,train_landmarks/450474571.parquet,450474571,2147437016,89,/kamo-vets/verse/sunset-park
13274,train_landmarks/450474571.parquet,450474571,2147452308,254,+55-86-44-54


In [17]:
# This a one sequence per file

train_df.groupby(by="sequence_id").sequence_id.count().reset_index(name="count").max()

sequence_id    2147465106
count                   1
dtype: int64

### each sequence has it's own file where we can spect several frames inside.

In [18]:
# how many do we have

train_df.shape

(67208, 5)

In [19]:
# how many phrases do we have?

train_df.phrase.nunique()

46478

# analisys

The problem consist in taking as input the sequence of differents hand landmarks and generating text base of that sequence.

Each landmark sequence has it's own parquet

Let's see the first one train_landmarks/5414471.parquet

In [20]:
landmark_sequence = pd.read_parquet(DATASET_DIR+os.sep+"train_landmarks/5414471.parquet").reset_index()

In [21]:
# I'm only interested in the hand coordinates, x and y

def get_columns(df):
    columns_right_hand = [column for column in df.columns if "x_right_hand" in column or "y_right_hand" in column]
    columns_left_hand = [column for column in df.columns if "x_left_hand" in column or "y_left_hand" in column]
    return columns_right_hand + columns_left_hand

In [22]:
hand_columns = get_columns(landmark_sequence)

In [23]:
np.save("feature_columms", np.array(hand_columns))

In [24]:
landmark_sequence.head()[hand_columns]

,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,x_right_hand_8,x_right_hand_9,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,0.566219,0.391196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,0.597674,0.441541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Sequences and frames.

selected_sequence = 1816796431
landmark_sequence[["sequence_id", "frame"]+hand_columns].query(f"sequence_id=={selected_sequence} and frame<20").sort_values(by="frame")

,sequence_id,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,1816796431,0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1816796431,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1816796431,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1816796431,3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1816796431,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1816796431,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1816796431,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1816796431,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1816796431,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1816796431,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# how many landmarks I have per sequence

sequences_pre_process_count_df = landmark_sequence.groupby(by="sequence_id").sequence_id.count().reset_index(name="count")

sequences_pre_process_count_df

,sequence_id,count
0,1816796431,123
1,1816825349,127
2,1816862427,127
3,1816909464,236
4,1816967051,102
...,...,...
995,1848141264,86
996,1848148361,184
997,1848157014,82
998,1848159052,212


# Preprocess

- remove all nan rows
- fill with 0 when there is no hand visible

In [29]:
from data_utils.preprocess import dropna, fillzero

In [30]:
landmark_sequence_new = dropna(landmark_sequence, hand_columns)
landmark_sequence_new = fillzero(landmark_sequence_new)

In [31]:
print(f"old shape: {landmark_sequence.shape} | new shape: {landmark_sequence_new.shape}")

old shape: (162699, 1631) | new shape: (95057, 1631)


In [37]:
### old

pd.merge(
sequences_pre_process_count_df,
landmark_sequence_new
    .drop_duplicates(subset=hand_columns)
    .groupby(by="sequence_id")
    .sequence_id.count()
    .reset_index(name="count_preprocessed"), on="sequence_id", how="inner")

,sequence_id,count,count_preprocessed
0,1816796431,123,58
1,1816825349,127,63
2,1816862427,127,127
3,1816909464,236,178
4,1816967051,102,56
...,...,...,...
995,1848141264,86,26
996,1848148361,184,150
997,1848157014,82,49
998,1848159052,212,83


In [45]:
# Why do we have sequences soo long??
# lets see 1816796431

print(f"phase: {train_df[train_df.sequence_id==1816796431].phrase}")

landmark_sequence_new[landmark_sequence_new.sequence_id==1816796431].drop_duplicates(subset=hand_columns)[["frame"]+hand_columns]

phase: 0    3 creekhouse
Name: phrase, dtype: object


,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,x_right_hand_8,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,0.566219,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,0.597674,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12,0.419290,0.509726,0.593165,0.685492,0.777913,0.483669,0.510993,0.536410,0.564583,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,0.398764,0.498118,0.583356,0.677779,0.775966,0.481279,0.491659,0.524974,0.571944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14,0.420213,0.495650,0.571790,0.659049,0.749740,0.485707,0.475930,0.501727,0.539150,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,15,0.386766,0.486770,0.571184,0.655207,0.737782,0.485260,0.487990,0.505752,0.526943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,16,0.416276,0.496218,0.567358,0.647024,0.728581,0.481693,0.490775,0.526558,0.572732,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,17,0.399879,0.483439,0.566848,0.648537,0.728934,0.482478,0.490363,0.531562,0.577581,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,18,0.395596,0.480669,0.559975,0.640045,0.722202,0.483103,0.488033,0.536422,0.590997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,19,0.405406,0.486883,0.562408,0.646358,0.724984,0.471614,0.502933,0.554993,0.595477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# so far 

# first: I get the right and left hand x and y coordinates.
# Drop the frames with no hands
# fill to zero when one hand is not in the sceen, this means the hand is not there. I will need to use a mask to ignore that.

In [26]:
def save_tfrecords(base_df):
    # TfRecord
    # Set length of frames to 128
    FRAME_LEN = 512
    FEATURE_COLUMNS = hand_columns

    # Create directory to store the new data
    if not os.path.exists(TFRECORDS_PATH+os.sep+"train_landmarks"):
        os.mkdir(TFRECORDS_PATH+os.sep+"train_landmarks")
    else:
        shutil.rmtree(TFRECORDS_PATH+os.sep+"train_landmarks")
        os.mkdir(TFRECORDS_PATH+os.sep+"train_landmarks")

    # Loop through each file_id
    for file_id in tqdm(base_df.file_id.unique(), total=len(base_df.file_id.unique())):
        # Parquet file name
        parquet_file = TRAIN_LANDMARKS_PATH+os.sep+f"{file_id}.parquet"

        # Sequences and frames inside the sequence for that file
        file_df = base_df.loc[base_df["file_id"] == file_id]

        # Fetch the parquet file
        df = pd.read_parquet(parquet_file).reset_index()[["sequence_id", "frame"]+FEATURE_COLUMNS]

        df = dropna(df, columns=FEATURE_COLUMNS)
        df = fillzero(df)
        df = df.sort_values(by=["sequence_id", "frame"])

        # print(df.shape)

        # File name for the updated data
        tf_file = TFRECORDS_PATH+os.sep+"train_landmarks"+os.sep+f"{file_id}.tfrecord"

        parquet_numpy = df.to_numpy()
        # Initialize the pointer to write the output of 
        # each `for loop` below as a sequence into the file.

        with tf.io.TFRecordWriter(tf_file) as file_writer:
            # Loop through each sequence in file.
            for seq_id, phrase in zip(file_df.sequence_id, file_df.phrase):
                # Fetch sequence data
                frames_df = df[df.sequence_id == seq_id].sort_values(by=["frame"])

                # to maintain order
                frames_numpy = frames_df[FEATURE_COLUMNS].to_numpy()

                #features dictionary
                # features = {FEATURE_COLUMNS[i]: tf.train.Feature(
                #     float_list=tf.train.FloatList(value=frames[:, i])) for i in range(len(FEATURE_COLUMNS))}

                #one feature all values from that sequence in order.
                features = {FEATURE_COLUMNS[i]: tf.train.Feature(
                    float_list=tf.train.FloatList(value=frames_numpy[:, i])) for i in range(len(FEATURE_COLUMNS))}            

                features["phrase"] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(phrase, 'utf-8')]))

                record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()

                file_writer.write(record_bytes)

In [27]:
# save_tfrecords(train_df)
FEATURE_COLUMNS = hand_columns

In [28]:
list_files = np.array([TFRECORDS_PATH+os.sep+"train_landmarks"+os.sep+file_name for file_name in os.listdir(TFRECORDS_PATH+os.sep+"train_landmarks/")])

In [29]:
list_files[:5]

array(['../data/asl-fingerspelling/tfrecords/train_landmarks/2026717426.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1726141437.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1664666588.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/638508439.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1497621680.tfrecord'],
      dtype='<U72')

In [30]:
import math

N = list_files.shape[0]*512

train_split = math.ceil(N*0.8)
validation_split = N - train_split

number_of_train_files = math.ceil(train_split/512)

np.random.shuffle(list_files)

train_files = list_files[:number_of_train_files]
val_files = list_files[number_of_train_files:]

print(f"train split: {train_files.shape[0]*512} | val split: {val_files.shape[0]*512}")

train split: 28160 | val split: 6656


In [31]:
train_files[:5]

array(['../data/asl-fingerspelling/tfrecords/train_landmarks/1099408314.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/882979387.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/105143404.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1365275733.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1255240050.tfrecord'],
      dtype='<U72')

In [32]:
train_dataset = tf.data.TFRecordDataset(train_files[0])

2023-08-15 10:53:26.094883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 10:53:26.113362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 10:53:26.113514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [33]:
next(iter(train_dataset))

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\x93\xc5\x01\n\xaa\x02\n\x0fy_right_hand_13\x12\x96\x02\x12\x93\x02\n\x90\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [101]:
import json

with open ("../data/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    char_to_num=json.load(f)

In [108]:
def decode_fn(record_bytes):
    schema = {COL: tf.io.VarLenFeature(dtype=tf.float32) for COL in FEATURE_COLUMNS}
    schema["phrase"] = tf.io.FixedLenFeature([], dtype=tf.string)
    features = tf.io.parse_single_example(record_bytes, schema)
    phrase = features["phrase"]
    landmarks = ([tf.sparse.to_dense(features[COL]) for COL in FEATURE_COLUMNS])
    # Transpose to maintain the original shape of landmarks data.
    landmarks = tf.transpose(landmarks)
    tf.reshape(landmarks, (-1, 84, 1))
    
    return landmarks, phrase

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=list(char_to_num.keys()),
        values=list(char_to_num.values()),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
)

# Add pad_token, start pointer and end pointer to the dict
pad_token = 'P'
start_token = '<'
end_token = '>'
pad_token_idx = 59
start_token_idx = 60
end_token_idx = 61

char_to_num[pad_token] = pad_token_idx
char_to_num[start_token] = start_token_idx
char_to_num[end_token] = end_token_idx
num_to_char = {j:i for i,j in char_to_num.items()}

# Reference: https://www.kaggle.com/code/irohith/aslfr-transformer/notebook

# Function to resize and add padding.
def resize_pad(x):
    if tf.shape(x)[0] < FRAME_LEN:
        x = tf.pad(x, ([[0, FRAME_LEN-tf.shape(x)[0]], [0, 0], [0, 0]]))
    else:
        x = tf.image.resize(x, (FRAME_LEN, tf.shape(x)[1]))
    return x

# Detect the dominant hand from the number of NaN values.
# Dominant hand will have less NaN values since it is in frame moving.
def pre_process(x):
    rhand = tf.gather(x, RHAND_IDX, axis=1)
    lhand = tf.gather(x, LHAND_IDX, axis=1)
    rpose = tf.gather(x, RPOSE_IDX, axis=1)
    lpose = tf.gather(x, LPOSE_IDX, axis=1)
    
    rnan_idx = tf.reduce_any(tf.math.is_nan(rhand), axis=1)
    lnan_idx = tf.reduce_any(tf.math.is_nan(lhand), axis=1)
    
    rnans = tf.math.count_nonzero(rnan_idx)
    lnans = tf.math.count_nonzero(lnan_idx)
    
    # For dominant hand
    if rnans > lnans:
        hand = lhand
        pose = lpose
        
        hand_x = hand[:, 0*(len(LHAND_IDX)//3) : 1*(len(LHAND_IDX)//3)]
        hand_y = hand[:, 1*(len(LHAND_IDX)//3) : 2*(len(LHAND_IDX)//3)]
        hand_z = hand[:, 2*(len(LHAND_IDX)//3) : 3*(len(LHAND_IDX)//3)]
        hand = tf.concat([1-hand_x, hand_y, hand_z], axis=1)
        
        pose_x = pose[:, 0*(len(LPOSE_IDX)//3) : 1*(len(LPOSE_IDX)//3)]
        pose_y = pose[:, 1*(len(LPOSE_IDX)//3) : 2*(len(LPOSE_IDX)//3)]
        pose_z = pose[:, 2*(len(LPOSE_IDX)//3) : 3*(len(LPOSE_IDX)//3)]
        pose = tf.concat([1-pose_x, pose_y, pose_z], axis=1)
    else:
        hand = rhand
        pose = rpose
    
    hand_x = hand[:, 0*(len(LHAND_IDX)//3) : 1*(len(LHAND_IDX)//3)]
    hand_y = hand[:, 1*(len(LHAND_IDX)//3) : 2*(len(LHAND_IDX)//3)]
    hand_z = hand[:, 2*(len(LHAND_IDX)//3) : 3*(len(LHAND_IDX)//3)]
    hand = tf.concat([hand_x[..., tf.newaxis], hand_y[..., tf.newaxis], hand_z[..., tf.newaxis]], axis=-1)
    
    mean = tf.math.reduce_mean(hand, axis=1)[:, tf.newaxis, :]
    std = tf.math.reduce_std(hand, axis=1)[:, tf.newaxis, :]
    hand = (hand - mean) / std

    pose_x = pose[:, 0*(len(LPOSE_IDX)//3) : 1*(len(LPOSE_IDX)//3)]
    pose_y = pose[:, 1*(len(LPOSE_IDX)//3) : 2*(len(LPOSE_IDX)//3)]
    pose_z = pose[:, 2*(len(LPOSE_IDX)//3) : 3*(len(LPOSE_IDX)//3)]
    pose = tf.concat([pose_x[..., tf.newaxis], pose_y[..., tf.newaxis], pose_z[..., tf.newaxis]], axis=-1)
    
    x = tf.concat([hand, pose], axis=1)
    x = resize_pad(x)
    
    x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
    x = tf.reshape(x, (FRAME_LEN, len(LHAND_IDX) + len(LPOSE_IDX)))
    return x

def convert_fn(landmarks, phrase):
    # Add start and end pointers to phrase.
    phrase = start_token + phrase + end_token
    phrase = tf.strings.bytes_split(phrase)
    phrase = table.lookup(phrase)
    # Vectorize and add padding.
    phrase = tf.pad(phrase, paddings=[[0, 64 - tf.shape(phrase)[0]]], mode = 'CONSTANT',
                    constant_values = pad_token_idx)

    # Apply pre_process function to the landmarks.
    # return pre_process(landmarks), phrase
    return landmarks, phrase

In [112]:
train_dataset = tf.data.TFRecordDataset(train_files).map(decode_fn).map(convert_fn).padded_batch(64)
val_dataset = tf.data.TFRecordDataset(val_files).map(decode_fn).map(convert_fn).padded_batch(16)

In [113]:
len(FEATURE_COLUMNS)

84

In [114]:
next(iter(train_dataset))

(<tf.Tensor: shape=(64, 403, 84), dtype=float32, numpy=
 array([[[0.        , 0.        , 0.        , ..., 0.5818536 ,
          0.5809219 , 0.5914433 ],
         [0.        , 0.        , 0.        , ..., 0.5647526 ,
          0.5552416 , 0.5568737 ],
         [0.        , 0.        , 0.        , ..., 0.55303997,
          0.5483256 , 0.5519165 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],
 
        [[0.239366  , 0.30775177, 0.3524068 , ..., 0.        ,
          0.        , 0.        ],
         [0.2571134 , 0.3253021 , 0.36963567, ..., 0.        ,
          0.        , 0.        ],
         [0.27146018, 0.3311777 , 0.3770892 , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.       